In [ ]:
#Install required libraries
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client
!pip install -q -U google-generativeai

In [ ]:
#Import necessary libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
# Set up your credentials and project details
# SERVICE_ACCOUNT_FILE = 'ANON.json'  # Upload this file to Colab
# PROJECT_ID = 'ANON'



In [ ]:
# Authenticate with the service account
# credentials = service_account.Credentials.from_service_account_file(
# SERVICE_ACCOUNT_FILE
# )

In [ ]:
import openpyxl
import google.generativeai as genai
from google.colab import userdata
import os
import json
from tqdm import tqdm

In [ ]:
# Import the API Key
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)



# Reading Excel Sheet

In [ ]:
path = "Explain.xlsx"
workbook = openpyxl.load_workbook(path)
sheet = workbook["All_data"]  # There is only one sheet, so no ambiguity here.

Get all the responses and scores

In [ ]:
INPUT_COLUMN = 8 # Specify the column from which input is read (A->1, B->2, etc., x->24)
APPLY_FILTER = True # Whether to filter the input rows
row_count =  1250 # sheet.max_row  # Get number of rows. If number of rows to read is already known (or sheet does not terminate where data terminates), place number here (+1) instead.
# Filter structure column number: ([values], valid/invalid)
#53: (["Mixed"], False),
INPUT_FILTER_DICTIONARY = { 4: ([
                                "3. Why do you think the approach you selected in the previous question will best support Jeremiah by assisting him with advocating for himself? <br/>",
                               "3. Why do you think the approach you selected in the previous question will best support Alexis by assisting her with advocating for herself? <br/>",
                               "11. Why do you think the approach you selected in the previous question will best support Alexis by assisting her with advocating for herself? <br/>",
                               "11. Why do you think the approach you selected in the previous question will best support Jeremiah by assisting him with advocating for himself? <br/>"],
                                True)}
# Function to check if a row matches the filter criteria
def matches_filters(sheet, row, filter_dict):
    for col, (valid_values, is_valid) in filter_dict.items():
        cell_value = sheet.cell(row=row, column=col).value
        if (cell_value in valid_values) != is_valid:
            return False
    return True

# Read the input data
inputs = []
input_rows = []
for i in range(2, row_count + 1):
    if not APPLY_FILTER or matches_filters(sheet, i, INPUT_FILTER_DICTIONARY):
        cell_value = sheet.cell(row=i, column=INPUT_COLUMN).value
        if cell_value is not None:
            inputs.append(cell_value)
            input_rows.append(i)

# Now `inputs` contains only the filtered data
#inputs = [sheet.cell(row=i, column=INPUT_COLUMN).value  for i in range(2, row_count + 1)]  # Column number is set here. YOUR INPUT COLUMN NUMBER GOES HERE.
#inputs = [x for x in inputs if x is not None]

Scoring Prompt Input

In [ ]:
SCORING_PROMPT_START = """
You are a tutor evaluator. Please assess a tutor’s response within a tutor training scenario involving a tutor instructing a middle school student to advocate for themselves when they are experiencing an inequity. Educators and tutors should apply the strategy of assisting students in identifying when they are experiencing an inequity and instruct them to advocate for themselves. The tutor is explaining the rationale behind their response. Assess and score the tutor’s response, as follows:


-if the tutor’s response demonstrates that they recognize that the student needs support in advocating for themselves and encourages the student to act, score with a 1. Sample responses scoring a 1 include: “It practices critical hope. It encourages her to advocate for herself by speaking to the teacher about it, and I offer my own help to assist her”; “It prepares Alexis to solve the problem herself by practicing in a low stress environment. It would make her more confident to talk to her teacher and do better in class.”


-if the tutor's response does not demonstrate that the tutor understands that the student needs support in advocating for themselves, score with a 0. Sample responses scoring a 0 include: “That would be the right approach because I am helping the student solve the problem rationally"; "This can let him feel supported and at the same time, gives him a solution of this problem"; "Because it teaches Alexis a possible way to avoid similar problems in the future; and "Discussing the plan with the student will provide them how to stand up for the inequity."


Response Start ---
"""

SCORING_FORMAT_PROMPT = """
--- Response End. Given the earlier response, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}.
"""

GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please assess a tutor’s response within a tutor training scenario involving a tutor instructing a middle school student to advocate for themselves when they are experiencing an inequity. Educators and tutors should apply the strategy of assisting students in identifying when they are experiencing an inequity and instruct them to advocate for themselves. The tutor is explaining the rationale behind their response. Assess and score the tutor’s response, as follows:

-if the tutor’s response demonstrates that they recognize that the student needs support in advocating for themselves and encourages the student to act, score with a 1. Sample responses scoring a 1 include: “It practices critical hope. It encourages her to advocate for herself by speaking to the teacher about it, and I offer my own help to assist her”; “It prepares Alexis to solve the problem herself by practicing in a low stress environment. It would make her more confident to talk to her teacher and do better in class.”

-if the tutor's response does not demonstrate that the tutor understands that the student needs support in advocating for themselves, score with a 0. Sample responses scoring a 0 include: “That would be the right approach because I am helping the student solve the problem rationally"; "This can let him feel supported and at the same time, gives him a solution of this problem"; "Because it teaches Alexis a possible way to avoid similar problems in the future; and "Discussing the plan with the student will provide them how to stand up for the inequity."


Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""


Feedback Elaboration Prompt

In [ ]:
# Second prompt: Pass feedback and the original input to the rewriter. Can omit the original input later to save on token cost.

FEEDBACK_PROMPT_START = """
Please write the following third-person feedback about a tutor into the second person. In doing so, please address the tutor directly and do not exceed 100 words.
Please start with positive feedback, if any. Subsequently, please provide any critiques, if applicable, in a constructive tone.

Third-person Feedback Start ---
"""

# Design choice: Passing original input to potentially provide more concrete second-person feedback (original feedback could be high-level). Can omit to reduce cost
ORIGINAL_RESPONSE_PROMPT = """
--- Third-person Feedback End. For further context, the original feedback was provided for the following response:

Response Start ---
"""

FEEDBACK_FORMAT_PROMPT_WITH_RESPONSE = """
--- Response End. Please return your refined feedback, based on the original response and the third-person feedback, in a JSON string following the format, {\"Feedback\": \"your response here\"}.
"""

FEEDBACK_FORMAT_PROMPT_WITHOUT_RESPONSE = """
--- Third-person Feedback End. Please return your refined feedback, based on the third-person feedback, in a JSON string following the format, {\"Feedback\": \"your response here\"}.
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## Gemini API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 300
TEMPERATURE =  2
RUN_UP_TO = 1250  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
TWO_STAGE = False  # Specifies whether to refine the feedback provided by the scoring prompt
TWO_STAGE_INCLUDE_RESPONSE = False # Specifies whether the second-stage prompt uses the original response.
SCORE_COLUMN = 16 # Change column numbers here to  modify where output is written
RATIONALE_COLUMN= 17
REFINED_RATIONALE_COLUMN= 18


MODEL = 'gemini-1.5-pro'
#'learnlm-1.5-pro-experimental'

model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)

# Add titling to the new columns
output_score_title = sheet.cell(row=1, column=SCORE_COLUMN)  # Output Score Column goes here. CHANGE COLUMN NUMBER to set score output location
output_score_title.value = "Gemini (few shot) Score, temp=2"  # OPTIONAL: Sets title cell in row 1. Comment out to disable this

rationale_title = sheet.cell(row=1, column=RATIONALE_COLUMN)
rationale_title.value = "Gemini (few shot) Rationale, temp=2"

if TWO_STAGE:  # Creates a new column for the refined rationale
  refined_rationale_title = sheet.cell(row=1, column=REFINED_RATIONALE_COLUMN)
  if TWO_STAGE_INCLUDE_RESPONSE:  # If the second-stage prompt also uses the original response, use a different column title
    refined_rationale_title.value = "Refined Feedback (w/ Response in prompt)"
  else:
    refined_rationale_title.value = "Refined Feedback (w/o response in prompt)"


if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
  input_rows_upto = input_rows[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
  input_rows_upto = input_rows

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
#build the prompt
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: " # Gemini Change
  generation_config = genai.GenerationConfig(max_output_tokens= MAX_TOKENS, temperature=TEMPERATURE) # TODO: Identify optimal temperature. take out # on left to set tempm make sure to remove the bracket as well to prevent a syntax error
  # Use client.chat.completions.create instead of client.generate_text
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config) # TODO: Potential break point

  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-5]
  #content = gemini_out.text # Assuming extract_response is defined correctly
  # We now need to parse the JSON into rational and score
  score_cell = sheet.cell(row=input_rows_upto[index], column=SCORE_COLUMN) # Score cell
  rationale_cell = sheet.cell(row=input_rows_upto[index], column=RATIONALE_COLUMN)  # Rationale cell
  refined_feedback_cell = sheet.cell(row=input_rows_upto[index], column=REFINED_RATIONALE_COLUMN)  # Refined feedback cell
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale

    score_cell.value = score  # Now write both into the Excel sheet
    rationale_cell.value = rationale

    # TODO: Update code to run two-stage
    if TWO_STAGE:  # If we are to refine the feedback
      if TWO_STAGE_INCLUDE_RESPONSE: # If we also want to pass the response, then we need to use a longer prompt, specified next:
        refinement_history = [{"role": "system", "content": FEEDBACK_PROMPT_START}, {"role": "user", "content": rationale}, {"role": "system", "content": ORIGINAL_RESPONSE_PROMPT}, {"role": "user", "content": inpt}, {"role": "system", "content":FEEDBACK_FORMAT_PROMPT_WITH_RESPONSE}]
      else: # If not, then we just pass the shorter prompt without the response
        refinement_history = [{"role": "system", "content": FEEDBACK_PROMPT_START}, {"role": "user", "content": rationale}, {"role": "system", "content":FEEDBACK_FORMAT_PROMPT_WITHOUT_RESPONSE}]
      # TODO: Switch this with Gemini call
      gemini_ss_out = client.chat.completions.create(model=MODEL, messages=refinement_history, max_tokens=MAX_TOKENS, temperature = TEMPERATURE)  # TODO: Use different model,  temperature, max token for the second stage
      role_ss, content_ss = extract_response(gemini_ss_out)  # ss: second stage
      try:
        refined_content_json = json.loads(content_ss)  #  Parse response into JSON
        refined_feedback = str(refined_content_json["Feedback"])
        refined_feedback_cell.value = refined_feedback  # Write the refined feedback into the excel sheet.
      except:
        refined_feedback_cell.value = "---"  # If Gemini / parsing fails for whatever reason.
  except:
    score_cell.value = "---"
    rationale_cell.value = "---"  # Failsafe
    if TWO_STAGE:
      refined_feedback_cell.value = "---"

## Save Excel Sheet

In [ ]:
FILE_NAME = "OutputSheet_max"+str(MAX_TOKENS)+"tokens_temp"+str(TEMPERATURE)  # Change file name here.
if RUN_UP_TO >= 0:
  FILE_NAME = FILE_NAME+"_first"+str(RUN_UP_TO)+"resp"
if TWO_STAGE:
  FILE_NAME = FILE_NAME+"_twostage"
if TWO_STAGE_INCLUDE_RESPONSE:
  FILE_NAME = FILE_NAME+"_include_resp_in_prompt"

FILE_NAME = FILE_NAME + ".xlsx"

workbook.save(FILE_NAME)  # Save the new file with OpenAI output into the file system